In [3]:
# load and optionally install required packages
if (!require('pacman')) install.packages('pacman')
pacman::p_load(
    dplyr,
    readr,
    stringr
)

In [ ]:
# initial read in of all data
csv_files <- list.files(path='./data/generated', full.names = T, pattern = 'data_with_tokenized*')
data_groups <- lapply(csv_files, read_csv)
data.raw <- do.call(rbind, data_groups)

csv_files <- list.files(path='./data/generated', full.names = T, pattern = 'cluster_keywords*')
data_groups <- lapply(csv_files, read_csv)
keywords.raw <- do.call(rbind, data_groups)

Rows: 10150 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 151348 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to


ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 26012 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_views...
lgl  (1): title

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 193368 Columns: 52
── Column specification ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (32): content, domain_url, images.url, lang, matched_profile, noise_cate...
dbl (19): engagement, fluency_level, porn_level, reach, sentiment, url_view

In [ ]:
keywords <- keywords.raw %>%
mutate(
    keywords = str_split(keywords, ' ')
)
data <- data.raw %>%
mutate(
    tokens = str_split(tokenized, ' ')
)


In [ ]:
keywords

In [ ]:
data

In [ ]:
data.guessed <- data %>%
merge(keywords, by.x = 'cluster', by.y = 'cluster_name') %>%
rowwise() %>%
mutate(
    matched_keywords = intersect(as.list(tokens), as.list(keywords)) %>% paste(collapse=','),
    matched_keywords_count = intersect(as.list(tokens), as.list(keywords)) %>% length()
) %>%
filter(matched_keywords_count > 0) %>%
group_by(content) %>%
top_n(1, matched_keywords_count) %>%
arrange(desc(matched_keywords_count)) %>%
select(content, keywords, tokens, matched_keywords, matched_keywords_count, cluster, cluster_number) %>%
mutate(
    id = paste0(cluster, '_', cluster_number)
)
data.guessed

In [ ]:
write_to_file = function(data.to_write){
    write_csv(data.to_write, paste0('./data/generated/tweets_by_cluster_', unique(data.to_write$id), '.csv'))
    return(data.to_write)
}

keywords %>%
mutate(
    id = paste0(cluster_name, '_', cluster_number)
) %>% merge(data.guessed) %>%
group_by(id) %>%
do(write_to_file(.))